In [53]:
import re

import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelBinarizer
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from train_utils import AverageMeter

In [54]:
match_cols = ['season', 'round'] + \
['date', 'time', 'referee', 'home_team', 'away_team', 'hom_score', 'away_score'] + \
['home_team_coach'] + \
['home_player_' + str(i) for i in range(1, 12)] + \
['home_substitute_' + str(i) for i in range(1, 8)] + \
['away_team_coach'] + \
['away_player_' + str(i) for i in range(1, 12)] + \
['away_substitute_' + str(i) for i in range(1, 8)]

In [55]:
train = pd.read_csv('train.csv')

In [56]:
train.head()

,season,round,year,month,day,hour,home_score,away_score,home_player_Sebastien Frey,home_player_Marco Amelia,...,away_coach_Roberto Mancini,away_coach_Rosario Pergolizzi,away_coach_Serse Cosmi,away_coach_Silvio Baldini,away_coach_Sinisa Mihajlovic,away_coach_Stefano Colantuono,away_coach_Stefano Pioli,away_coach_Walter Alfredo Novellino,away_coach_Walter Mazzarri,away_coach_Walter Zenga
0,0,1,2005,8,27,20,2,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,2005,8,27,18,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2005,8,28,15,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2005,8,28,15,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# utility methods
def get_column_names_containing_str(df: pd.DataFrame, substring: str) -> list[str]:
    return df.loc[:, df.columns.str.contains(substring)].columns.values.tolist()


def get_team_and_historical_index_from_match_team_id(match_team_id: str) -> (str, str):
    match_team_name = re.findall("\s+", match_team_id)[0]
    match_team_index = re.findall("\d+", match_team_id)[0]
    return match_team_name, match_team_index


def get_match_by_team_season_round(df: pd.DataFrame, team: str, season: int, round: int) -> pd.DataFrame:
    return df[((df[f'home_team_{team}'] == 1) | (df[f'away_team_{team}'] == 1)) & (df['round'] == round) & (df['season'] == season)]


def get_last_n_matches_played_by_team_before_round_in_season(df: pd.DataFrame, team: str, season: int, round: int, n: int) -> pd.DataFrame:
    last_n_matches = pd.DataFrame()
    for i in range(1, n + 1):
        if round - i > 0:
            last_n_matches = pd.concat([last_n_matches, get_match_by_team_season_round(df, team, season, round - i)])
    return last_n_matches


def fill_with_padding(source: pd.DataFrame):
    if len(source) < 5:
        initial_len = len(source)
        padding = source.tail(1)
        for i in range(5 - len(source)):
            source = pd.concat([source, padding], ignore_index=True)
        # print(f'padding applied. Initial len: {initial_len} new_len: {len(source)}')
    return source


def get_home_team_name(row: pd.DataFrame) -> str:
    result = row.loc[:, [col for col in row.columns if col.startswith('home_team_')]]
    print(result)

In [58]:
get_home_team_name(train)

      home_team_ASCOLI  home_team_ATALANTA  home_team_BOLOGNA  \
0                    0                   0                  0   
1                    0                   0                  0   
2                    1                   0                  0   
3                    0                   0                  0   
4                    0                   0                  0   
...                ...                 ...                ...   
1307                 0                   0                  0   
1308                 0                   1                  0   
1309                 0                   0                  0   
1310                 0                   0                  0   
1311                 0                   0                  1   

      home_team_CAGLIARI  home_team_CATANIA  home_team_CHIEVOVERONA  \
0                      0                  0                       0   
1                      0                  0                       0   
2     

In [59]:
tot_num_of_feats = len(train.columns)
print(f'Total number of encoded features: {tot_num_of_feats}')

Total number of encoded features: 2590


In [60]:
del train
# del lb

### Dataset construction

We need to define a torch Dataset and torch Dataloader that will be used during training.

In [61]:
class SerieAFootballMatchesDataset(Dataset):
    history_len = 5

    def __init__(self, csv_file):
        self.dataframe = pd.read_csv(csv_file)

    def __len__(self) -> int:
        return len(self.dataframe)

    def __getitem__(self, idx):
        # print(f'picked index {idx}')
        idx = self.scale_idx(idx)
        x_as_df = self.dataframe.iloc[[idx]]  # df
        x = self.dataframe.loc[idx]  # series
        # encoder expects a dataframe with a 'result' column
        y = self.dataframe[['result_home', 'result_draw', 'result_home']]
        last_n_games_home, last_n_games_away = self.retrieve_historical_data(x)
        last_n_games_home = fill_with_padding(last_n_games_home)
        last_n_games_away = fill_with_padding(last_n_games_away)
        x, x_historical_home, x_historical_away, y = self.to_tensor(x_as_df, last_n_games_home, last_n_games_away, y)
        return x, x_historical_home, x_historical_away, y

    def scale_idx(self, idx: int) -> int:
        """Scale the given index to a range that allows for historical data retrieval"""
        old_min = 0
        old_max = len(self.dataframe)
        '''
        idx = 10 corresponds to the first match of the second round.
        This ensure the retrieval of at least 1 historical match.
        In the worst case scenario, padding will fill the other 4 historical slots.
        '''
        new_min = 10
        new_max = old_max
        old_range = old_max - old_min
        new_range = new_max - new_min
        normalized_idx = (idx - old_min) / old_range
        return int(round(normalized_idx * new_range + new_min))

    def retrieve_historical_data(self, source: pd.DataFrame):
        """Retrieve historical data for home and away teams from source"""
        last_n_games_home = get_last_n_matches_played_by_team_before_round_in_season(
            self.dataframe, source['home_team'], source['season'], source['round'], self.history_len)
        last_n_games_away = get_last_n_matches_played_by_team_before_round_in_season(
            self.dataframe, source['away_team'], source['season'], source['round'], self.history_len)
        return last_n_games_home, last_n_games_away

    def to_tensor(self, x: pd.DataFrame, x_historical_home: pd.DataFrame, x_historical_away: pd.DataFrame,
               y: pd.DataFrame):
        x_tensor = torch.tensor(x.values)
        x_historical_home_tensor = torch.tensor(x_historical_home)
        x_historical_away_tensor = torch.tensor(x_historical_away)
        y_tensor = torch.tensor(y)
        return x_tensor, x_historical_home_tensor, x_historical_away_tensor, y_tensor


## Training

In [62]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh = nn.Tanh()

    def forward(self, input, hidden):
        combined = torch.cat([input, hidden], dim=0)
        pre_hidden = self.linear(combined)
        hidden = self.tanh(pre_hidden)
        return hidden

    def init_hidden(self, minibatch_size):
        return torch.zeros(minibatch_size, self.hidden_size)


class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
            # nn.Softmax(dim=1) softmax is applied implicitly by CrossEntropyLoss
        )

    def forward(self, x):
        # 'x' is the combination of: 'x', 'x_historical_home', 'x_historical_away'
        # they all have size: minibatch_size x num_of_feats
        x = self.flatten(x) # just in case x was not flattened
        output = self.layers(x)
        return output


class HybridNetwork(nn.Module):
    def __init__(self, rnn_home_model: RNN, rnn_away_model: RNN, mlp_model: NeuralNetwork):
        super(HybridNetwork, self).__init__()
        self.rnn_home = rnn_home_model
        self.rnn_away = rnn_away_model
        self.mlp = mlp_model

    def forward(self, x, x_historical_home, x_historical_away):
        # 'x' comes in as:                minibatch_size x 1 x num_of_feats
        # 'x_historical_*' comes in as:   minibatch_size x 5 x num_of_feats
        # 'rnn_*_hidden' will be:         minibatch_size x num_of_feats
        batch_size = x.size(0)
        time_seq_len = x_historical_home.size(1)
        # RNN HOME FORWARD
        rnn_home_hidden = self.rnn_home.init_hidden(batch_size)
        # print(rnn_home_hidden)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_home_hidden[batch_idx] = self.rnn_home(
                    torch.flatten(x_historical_home[batch_idx, history_idx]),
                    rnn_home_hidden[batch_idx])
        # RNN AWAY FORWARD
        rnn_away_hidden = self.rnn_away.init_hidden(batch_size)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_away_hidden[batch_idx] = self.rnn_away(
                    torch.flatten(x_historical_away[batch_idx, history_idx]),
                    rnn_away_hidden[batch_idx])
        # print(rnn_home_hidden)
        # MLP FORWARD
        # concat on the features dimension
        x_train = torch.cat([x, rnn_home_hidden, rnn_away_hidden], dim=1)
        print(f'x_train shape: {x_train.shape}')
        y_hat = self.mlp(x_train)
        return y_hat

In [63]:
def train_epoch(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, loss_meter):
    for x, x_historical_home, x_historical_away, y in dataloader:
        optimizer.zero_grad()
        y_hat = model(x, x_historical_home, x_historical_away)
        # print(f'y: {y}')
        # print(f'y_hat: {y_hat}')
        loss = loss_fn(y.to(dtype=torch.float), y_hat)
        print(f'loss: {loss}')
        loss.backward()
        optimizer.step()
        loss_meter.update(val=loss.item(), n=x.shape[0])


def train_model(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, num_epochs: int):
    model.train()
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        train_epoch(model=model, dataloader=dataloader, optimizer=optimizer, loss_fn=loss_fn, loss_meter=loss_meter)
        print(f"Epoch {epoch + 1} completed. Training loss: {loss_meter.avg}")

In [64]:
train_dataset = SerieAFootballMatchesDataset(csv_file='train.csv')

In [65]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [66]:
learning_rate = 0.001
num_epochs = 3
hidden_size = 128
rnn_home = RNN(input_size=tot_num_of_feats, hidden_size=hidden_size)
rnn_away = RNN(input_size=tot_num_of_feats, hidden_size=hidden_size)
# we have two hidden states (for home and away team) plus all features except for 'home_score', 'away_score', 'result_home', 'result_draw' and 'result_away'
mlp = NeuralNetwork(hidden_size * 2 + tot_num_of_feats - 5)
model = HybridNetwork(rnn_home_model=rnn_home, rnn_away_model=rnn_away, mlp_model=mlp)
cross_entropy_loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [67]:
train_model(model=model, dataloader=train_dataloader, optimizer=optimizer, loss_fn=cross_entropy_loss_fn, num_epochs=num_epochs)

KeyError: 'home_team'

# Missing data
- We don't have data about new players that come to play in _Serie A_ during the course of the seasons. The model has to learn from zero context how important their contribution is for the outcome of the matches. If we were to considered multiple leagues, we could keep track of player transfers and maintain the history.
- We don't have data about cup matches played during the course of the seasons, like _Champions League_, _Europa League_ and _Coppa Italia_. Since they are very prestigious competitions and matches are usually very competitive, teams put a lot of effort in them and therefore can then perform worse in the championship.
- We don't have any type of player performance metric like who scored a goal, who was the assist man, red or yellow cards, goalkeeper's saves etc. so the model could face some difficulties in learning which player is important for the team.